In [35]:
import numpy as np
from frappy.datatypes import (
    ArrayOf,
    BoolType,
    DataType,
    EnumType,
    FloatRange,
    IntRange,
    ScaledInteger,
    StringType,
    StructOf,
    TupleOf,
    BLOBType,
)






secopdtype = StructOf(
         red = IntRange(0,255),
         green = IntRange(0,255),
         blue = IntRange(0,255),
         nested = StructOf(
             name = StringType(),
             age = IntRange()
            ),
         tuple_nest = TupleOf(
             FloatRange(),
             IntRange(),
             StringType()
         )
         )





elem = {'red':25,'green':13,'blue':100,'nested':{'name':'peter','age':24},'tuple_nest':(14.4,7,'test')}



my_dict = {'a': [1, 2, 3], 'b': {'b1': [4, 5, 6], 'b2': {'c': [7, 8, 9]}}}




keys = elem.keys()
items = elem.items()


#print(list(elem.keys()))
#secopdtype.check_type(elem)



#an = np.array(object=[1,1,1],dtype=dtype_obj)




SECOP2PY = {
    IntRange: int,
    FloatRange: float,
    StringType: 'U100',
    BoolType: bool,
    BLOBType: 'U100',
    ScaledInteger: int,
    EnumType: int,
} 


dt = np.dtype(
    [
        (
            "a",
            [
                ("b", int),
                ("c", 'U10'),
            ],
        ),
        ("d", float),
    ]
)




obj_elem ={'a':{'b':12,'c':'test'},'d':1.4} 


print(dt.descr)

def mkdt(secop_dt:DataType):
    dt_list = []
    if isinstance(secop_dt,StructOf):
        for key,secop_dtype in secop_dt.members.items():
            if isinstance(secop_dtype,(StructOf,TupleOf)):
                dt_list.append((key,mkdt(secop_dtype)))
                continue
                
            
            dt_list.append((key,SECOP2PY[secop_dtype.__class__]))
            
    if isinstance(secop_dt,TupleOf):
        for index,secop_dtype in enumerate(secop_dt.members):
            if isinstance(secop_dtype,(StructOf,TupleOf)):
                dt_list.append(('idx'+str(index),mkdt(secop_dtype)))
                continue
                
            
            dt_list.append(('idx'+str(index),SECOP2PY[secop_dtype.__class__]))
        
    return dt_list


print(mkdt(secopdtype))

np.dtype(mkdt(secopdtype))


def mklist(secop_value:dict|tuple,secop_dt:DataType) -> list:
   
    
    if isinstance(secop_dt,StructOf):
        ret_val = tuple([ mklist(secop_value[key],member_dt) for key, member_dt in secop_dt.members.items() ])
        
    elif isinstance(secop_dt,TupleOf):
        ret_val = tuple([ mklist(secop_value[idx],member_dt) for idx, member_dt in enumerate(secop_dt.members)])
    
    else:
        ret_val = secop_value

    return ret_val

#print(secopdtype.check_type(elem))

converted_elem = mklist(elem,secopdtype)

print(converted_elem)

retarr = np.asarray(a=mklist(elem,secopdtype),dtype=mkdt(secopdtype))
print(retarr['nested'])
print(retarr)
#arr =np.asarray(elem,dtype=object)



a = ((20,'test'),1.23)

npa = np.asarray(a,dt)

print(npa['a'])


[('a', [('b', '<i8'), ('c', '<U10')]), ('d', '<f8')]
[('red', <class 'int'>), ('green', <class 'int'>), ('blue', <class 'int'>), ('nested', [('name', 'U100'), ('age', <class 'int'>)]), ('tuple_nest', [('idx0', <class 'float'>), ('idx1', <class 'int'>), ('idx2', 'U100')])]
(25, 13, 100, ('peter', 24), (14.4, 7, 'test'))
('peter', 24)
(25, 13, 100, ('peter', 24), (14.4, 7, 'test'))
(20, 'test')


In [ ]:
import numpy as np

# Define a sample nested dictionary
my_dict = {'a': [1, 2, 3], 'b': {'b1': [4, 5, 6], 'b2': {'c': [7, 8, 9]}}}

# Define a recursive function to convert the nested dictionary to a structured array
def dict_to_structured_array(d):
    if isinstance(d, dict):
        return np.core.records.fromarrays([dict_to_structured_array(v) for v in d.values()], dtype=[(k, dict_to_structured_array(v).dtype) for k, v in d.items()])
    else:
        return np.array(d)

# Convert the nested dictionary to a structured array
my_array = dict_to_structured_array(my_dict)

print(my_array)
print(my_array.dtype)

[(1, (4, (7,))) (2, (5, (8,))) (3, (6, (9,)))]
(numpy.record, [('a', '<i8'), ('b', (numpy.record, [('b1', '<i8'), ('b2', (numpy.record, [('c', '<i8')]))]))])


In [17]:
import numpy as np

arr = (12.1,[1,2,2])

dt = [('flt', float,[1]), ('testarr', int, [3])]

np_arr = np.asarray(a=arr,dtype=dt)

print(np_arr)



([12.1], [1, 2, 2])
